<a href="https://colab.research.google.com/github/satani99/generative_deep_learning/blob/main/02_02_deep_learning_convolutions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch.nn as nn
import torch 

convlayer1 = nn.Conv2d(1, 2, kernel_size=3)

In [4]:
class Conv(nn.Module):
  def __init__(self):
    super(Conv, self).__init__()

    self.conv_layer_1 = nn.Conv2d(3, 10, kernel_size=4, stride=2)
    self.conv_layer_2 = nn.Conv2d(10, 20, kernel_size=3, stride=2)  
    self.linear = nn.Linear(20*8*8, 10)

  def forward(self, x):
    x = self.conv_layer_1(x)
    x = self.conv_layer_2(x)
    x = torch.flatten(x, 1)
    output = F.softmax(self.linear(x))
    return output

In [5]:
net = Conv()
print(net)

Conv(
  (conv_layer_1): Conv2d(3, 10, kernel_size=(4, 4), stride=(2, 2))
  (conv_layer_2): Conv2d(10, 20, kernel_size=(3, 3), stride=(2, 2))
  (linear): Linear(in_features=1280, out_features=10, bias=True)
)
